In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import os
import pandas as pd
%matplotlib inline
from dask.diagnostics import ProgressBar

In [ ]:
# Uncomment next line for cheyenne:
#dir_base = '/glade/p/CMIP/CMIP5/output1/'
dir_base = './'

center = "NCAR"; model = "CCSM4"
scenario = "historical"
freq = "6hr"; realm = "atmos"; dtype = "6hrLev"

ddir = dir_base+center+'/'+model+'/'+scenario+'/'+freq+'/'+realm+'/'+dtype+'/'

# loop over ensemble members
for ens in os.listdir(ddir):
# lets pick a year for testing
    syear = str(1960)
    sdir = ddir+ens+'/latest/'
    tdir = '_6hrLev_'+model+'_'+scenario+'_'+ens+'_'
# all variables in all files for this year:
    fullyear = sdir+'*/*'+tdir+syear+'*.nc'
    with ProgressBar():
        ds = xr.open_mfdataset(fullyear)
        qu = ds.hus*ds.ua
        qv = ds.hus*ds.va
        dsm = ds.groupby('time.month').mean('time')
        dsm['qum']= qu.groupby('time.month').mean('time') - dsm.hus * dsm.ua
        dsm['qvm']= qv.groupby('time.month').mean('time') - dsm.hus * dsm.va
        dsm.to_netcdf('./output/monthly-'+ens+'.nc')
    

[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[###############################         ] | 79% Completed | 18.6s

In [4]:
dsm.month
# how to change time coordinate to datetime? preserve the year?

<xarray.DataArray 'month' (month: 12)>
array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])
Coordinates:
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12

In [7]:
"""  other questions:
1. how to remove a dataarray from a dataset?
2. how to get the monthly mean to be at middle of month:
 for example, create a fake time series and create monthly mean 
    and more generally
3. how to replace coordinates of a dimension
"""

rng = pd.date_range('1979-01-01T00:00:00', periods=365, freq='D')
ts = pd.Series(np.random.randn(len(rng)), index=rng)
ts.resample('M').mean()


1979-01-31    0.130752
1979-02-28   -0.163251
1979-03-31    0.192035
1979-04-30    0.040189
1979-05-31   -0.162213
1979-06-30   -0.091121
1979-07-31    0.177308
1979-08-31   -0.026541
1979-09-30   -0.009021
1979-10-31    0.566593
1979-11-30   -0.061222
1979-12-31    0.126624
Freq: M, dtype: float64